In [5]:
import pandas as pd
import numpy as np
import torch 
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
import prelim_process_df
from datetime import timedelta, datetime
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader

In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
data_path = '/home/jwelliav/Documents/Crypto/data/Binance/'

path_avax = data_path + 'AVAX_USDT.csv'

path_eth = data_path + 'ETH_USDT.csv'

path_btc = data_path + 'BTC_USDT.csv'

#path_luna = data_path + 'LUNA_USDT.csv'

path_sol = data_path + 'SOL_USDT.csv'

In [4]:
"""
Drop the returns,log_returns cols
"""

def drop_ret_cols(df):
    b = []
    for x in df.columns:
        if x[:3] == 'ret' or x[:3] == 'log':
            b.append(x)
    return(df.drop(columns = b)) 

In [5]:
                                                                                                                                                                                                                                                                        
df_eth =   drop_ret_cols(prelim_process_df.read_process_df(path_eth,'eth'))

df_btc =   drop_ret_cols(prelim_process_df.read_process_df(path_btc,'btc'))

#df_luna =  drop_ret_cols(prelim_process_df.read_process_df(path_luna,'luna'))

df_sol =   drop_ret_cols(prelim_process_df.read_process_df(path_sol,'sol'))

In [6]:
df_avax.head()

,open_avax,high_avax,low_avax,close_avax,volume_avax,vol_usdt_K_avax
datetime,,,,,,
2020-09-22 15:31:00,3.6018,4.4804,3.5897,4.2775,295523.27,1164.258251
2020-09-22 15:32:00,4.2500,4.4351,4.0611,4.1021,137106.97,572.565562
2020-09-22 15:33:00,4.1027,4.2000,4.0061,4.1699,117775.14,487.153312
2020-09-22 15:34:00,4.1699,4.1990,4.1076,4.1800,72788.25,303.887304
2020-09-22 15:35:00,4.1704,4.3879,4.0487,4.3350,115621.12,491.701937


In [7]:
assets_dict = {}
assets_dict['avax'] = df_avax
assets_dict['eth'] = df_eth
assets_dict['btc'] = df_btc
#assets_dict['luna'] = df_luna
assets_dict['sol'] = df_sol

In [8]:
"""
Merge all the dataframes into one 
"""

df_abes = df_avax.copy()

for k in assets_dict.keys():
    if k != 'avax':
        df_abes = pd.merge(df_abes,assets_dict[k],left_index = True,right_index = True)       

In [9]:
"""
keep only the close price columns in a copy df_abes and the resample to 30 mins intervals.  
"""

b = []
for x in df_abes.columns:
    if x[:3] == 'ope' or x[:3] == 'hig' or x[:3] == 'low':
        b.append(x)
        
        
df_abes_cls = df_abes.drop(columns = b).copy() 

df_abes_cls = prelim_process_df.resample(df_abes_cls,'30T')

## Missing values

In [81]:
s = df_abes_cls.isna().sum(axis = 1)

s = s > 0

In [82]:
# number of NaN values

s.sum()

42

In [83]:
bad_indices = []

for i in s.index:
    if s[i] == True:
        bad_indices.append(i)        

In [84]:
df_abes_cls.loc[bad_indices]

,close_avax,volume_avax,vol_usdt_K_avax,close_eth,volume_eth,vol_usdt_K_eth,close_btc,volume_btc,vol_usdt_K_btc,close_sol,volume_sol,vol_usdt_K_sol
datetime,,,,,,,,,,,,
2020-11-30 15:00:00,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0
2020-11-30 15:30:00,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0
2020-12-21 23:30:00,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0
2020-12-22 00:00:00,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0
2020-12-22 00:30:00,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0
2020-12-22 01:00:00,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0
2020-12-22 01:30:00,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0
2020-12-22 02:00:00,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0
2020-12-22 02:30:00,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0


In [85]:
df_abes_cls.isna().sum()

close_avax         42
volume_avax         0
vol_usdt_K_avax     0
close_eth          42
volume_eth          0
vol_usdt_K_eth      0
close_btc          42
volume_btc          0
vol_usdt_K_btc      0
close_sol          42
volume_sol          0
vol_usdt_K_sol      0
dtype: int64

In [86]:
for x in df_abes_cls.columns:
    if x[:3] == 'vol':
        bad_indices += list(df_abes_cls[df_abes_cls[x] == 0].index)    
        
bad_indices = list(set(bad_indices))        

In [87]:
len(bad_indices)

47

In [88]:
df_abes_cls.head()

,close_avax,volume_avax,vol_usdt_K_avax,close_eth,volume_eth,vol_usdt_K_eth,close_btc,volume_btc,vol_usdt_K_btc,close_sol,volume_sol,vol_usdt_K_sol
datetime,,,,,,,,,,,,
2020-09-22 15:30:00,4.8811,2808316.25,13498.099902,342.05,20140.99740,6886.245163,10423.27,1305.659365,13609.113154,2.8000,61420.44,172.465165
2020-09-22 16:00:00,4.9312,1914636.29,9136.249619,339.99,23133.33317,7898.110882,10412.11,1147.850269,11969.341525,2.7540,53219.28,148.122713
2020-09-22 16:30:00,4.9096,1256377.31,6213.493286,336.12,48515.72194,16362.884815,10372.90,1801.381449,18703.921799,2.6728,158269.17,425.511003
2020-09-22 17:00:00,5.3951,1100528.85,5757.581164,339.58,32510.29604,11019.057797,10429.77,1388.738454,14468.525827,2.8134,267098.29,742.661738
2020-09-22 17:30:00,6.8219,2783452.58,17595.535036,342.11,23494.62347,8002.336092,10463.18,1288.599265,13468.384675,2.7602,95408.49,263.655611


In [89]:
#import plotly.express as px
#
#fig = px.line(df_abes_cls,y= ["close_avax",'close_sol'])
#
#fig.update_xaxes(
#    dtick="M1",
#    tickformat="%b\n%Y")
#fig.show()
#

### getting those indices which don't have the full 30 minutes data

In [90]:
len(df_abes_cls)

24953

In [91]:
idx_30t = list(df_abes_cls.index)
idx = list(df_abes.index)

idx_int = list(set(idx).intersection(set(idx_30t)))

idx_int.sort()

In [92]:
ok_indices = {}

l_g = list(set(idx_30t) - set(idx_int))



#for x in l_g:
#    ok_indices[x] = 0

In [93]:
bad_indices += l_g

bad_indices = list(set(bad_indices))

In [94]:
j = 0

idx_match = {}

for i in range(len(idx)):
    if j < len(idx_int):
        current = idx_int[j]
    if current == idx[i]:
        idx_match[current] = i
        j += 1      

In [95]:
ok_indices = {}

for i in range(len(idx_int)-1):
    x = idx_int[i]
    a = idx_match[x]
    b = idx_match[idx_int[i+1]]
    if b-a == 30:
        ok_indices[x] = 1
    else:
        ok_indices[x] = 0

In [96]:
#bad_indices = []

for x in idx_30t:
    if x in ok_indices.keys():
        if ok_indices[x] == 0:
            bad_indices.append(x + timedelta(minutes = 30))
    else:
        bad_indices.append(x + timedelta(minutes = 30))

In [97]:
bad_indices = list(set(bad_indices))

len(bad_indices)

60

In [98]:
bad_indices.sort()
#bad_indices

In [99]:
bad_indices[3]-bad_indices[1]

Timedelta('68 days 23:30:00')

In [100]:
key_bad = []
current = 0
key_bad.append(bad_indices[current])

d = {}
d[bad_indices[current]] = []
d[bad_indices[current]].append(bad_indices[current])

i = 1

while i <= len(bad_indices)-1:
        if bad_indices[i] - bad_indices[current] == timedelta(minutes = 30*(i-current)):
            d[bad_indices[current]].append(bad_indices[i])              
            i += 1
        else:
            current = i
            key_bad.append(bad_indices[current])
            d[bad_indices[current]] = [bad_indices[current]]
            i += 1

In [101]:
timedists_kb = []

for i in range(len(key_bad)):
    for j in range(i+1,len(key_bad)):
         timedists_kb.append(key_bad[j]-key_bad[i])

min(timedists_kb)

Timedelta('3 days 12:30:00')

In [102]:
#reversing the dict

r_d = {}

for k in d:
    for v in d[k]:
        r_d[v] = k

In [103]:
#df_abes_cls.loc[l]

In [104]:
l = list(d.keys())
l.sort()

for x in l[1:]:
    l_r = [x - timedelta(minutes = i*30) for i in range(1,10)]
    t = df_abes_cls.loc[l_r].mean()
    tmp = d[x]
    for a in tmp:
        df_abes_cls.loc[a] = t

In [106]:
#df_abes_cls.to_csv('../../data/data_for_models/lstm_30m.csv')

In [10]:
df_abes_cls = pd.read_csv('../../data/data_for_models/lstm_30m.csv')

In [11]:
df_abes_cls.head()

,datetime,close_avax,volume_avax,vol_usdt_K_avax,close_eth,volume_eth,vol_usdt_K_eth,close_btc,volume_btc,vol_usdt_K_btc,close_sol,volume_sol,vol_usdt_K_sol
0,2020-09-22 15:30:00,4.8811,2808316.25,13498.099902,342.05,20140.99740,6886.245163,10423.27,1305.659365,13609.113154,2.8000,61420.44,172.465165
1,2020-09-22 16:00:00,4.9312,1914636.29,9136.249619,339.99,23133.33317,7898.110882,10412.11,1147.850269,11969.341525,2.7540,53219.28,148.122713
2,2020-09-22 16:30:00,4.9096,1256377.31,6213.493286,336.12,48515.72194,16362.884815,10372.90,1801.381449,18703.921799,2.6728,158269.17,425.511003
3,2020-09-22 17:00:00,5.3951,1100528.85,5757.581164,339.58,32510.29604,11019.057797,10429.77,1388.738454,14468.525827,2.8134,267098.29,742.661738
4,2020-09-22 17:30:00,6.8219,2783452.58,17595.535036,342.11,23494.62347,8002.336092,10463.18,1288.599265,13468.384675,2.7602,95408.49,263.655611


#### Obtain percentage change for closed price columns


In [132]:
df_abes_cls = pd.read_csv('../../data/data_for_models/lstm_30m.csv',parse_dates = True,index_col = 'datetime')

In [133]:
#df_abes_cls = pd.read_csv('../../data/data_for_models/lstm_30m.csv')
#
#df_abes_cls.index = df_abes_cls.datetime
#df_abes_cls.index = pd.to_datetime(df_abes_cls.index)
#
#df_abes_cls.drop('datetime',inplace = True, axis = 1)

In [15]:
close_cols = []

for c in df_abes_cls.columns:
    if c[:3] == 'clo':
        close_cols.append(c)

for c in close_cols:
    df_abes_cls['returns' + c[5:]] = df_abes_cls[c].pct_change()

In [16]:
#datetime.strptime(df_abes_cls.index[0],'%Y-%m-%d %H:%M:%S')

In [17]:
df_abes_cls = df_abes_cls.dropna(axis=0, how='any')


In [18]:
df_abes_cls.index[0]

Timestamp('2020-09-22 16:00:00')

In [19]:
a = [x + timedelta(days = 1) for x in df_abes_cls.index]
b = [x + timedelta(days = 2) for x in df_abes_cls.index]
c = [x + timedelta(days =3) for x in df_abes_cls.index]

In [20]:
final_id = df_abes_cls.index[-1]

In [21]:
a = a[:list(a).index(final_id)]
b = b[:list(b).index(final_id)]
c = c[:list(c).index(final_id)]

### Getting labels 

In [22]:
def get_labels(df,c,days): 
    
    l = df.index
    final_id = l[-1]
    
    labels = {}
   
    for x in l:
        tmp1 = x + timedelta(days = days)
        if tmp1 < final_id:
            if (df.loc[tmp1][c] - df.loc[x][c])/(df.loc[x][c] + 0.00001) > 0.05:
                labels[x] = 1
            else:
                labels[x] = 0
        else:
            labels[x] = -1
    return(labels)           

In [23]:
for c in df_abes_cls.columns:
    if c[:3] == 'clo':
        for i in [1,2,3]:
            a_tmp = pd.Series(get_labels(df_abes_cls,c,i))
            c_tmp = c.split('close_')[1] + '_label_' + str(i)
            df_abes_cls[c_tmp] = a_tmp 

In [24]:
df_abes_cls

,close_avax,volume_avax,vol_usdt_K_avax,close_eth,volume_eth,vol_usdt_K_eth,close_btc,volume_btc,vol_usdt_K_btc,close_sol,...,avax_label_3,eth_label_1,eth_label_2,eth_label_3,btc_label_1,btc_label_2,btc_label_3,sol_label_1,sol_label_2,sol_label_3
datetime,,,,,,,,,,,,,,,,,,,,,
2020-09-22 16:00:00,4.931200,1.914636e+06,9136.249619,339.990000,23133.333170,7898.110882,10412.110000,1147.850269,11969.341525,2.7540,...,0,0,0,0,0,0,0,0,1,1
2020-09-22 16:30:00,4.909600,1.256377e+06,6213.493286,336.120000,48515.721940,16362.884815,10372.900000,1801.381449,18703.921799,2.6728,...,0,0,0,0,0,0,0,0,1,1
2020-09-22 17:00:00,5.395100,1.100529e+06,5757.581164,339.580000,32510.296040,11019.057797,10429.770000,1388.738454,14468.525827,2.8134,...,0,0,0,0,0,0,0,0,1,1
2020-09-22 17:30:00,6.821900,2.783453e+06,17595.535036,342.110000,23494.623470,8002.336092,10463.180000,1288.599265,13468.384675,2.7602,...,0,0,0,0,0,0,0,0,1,1
2020-09-22 18:00:00,6.448800,2.538824e+06,16273.012136,340.500000,18641.707730,6364.471484,10445.040000,990.685762,10355.010636,2.7265,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-24 10:00:00,73.280000,8.301071e+04,6109.125097,2550.310000,13215.990300,33837.032941,36702.300000,1095.436680,40307.563870,84.3000,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2022-02-24 10:30:00,74.300000,6.866388e+04,5086.607976,2570.800000,10013.189800,25674.415957,36949.630000,900.574340,33227.209216,84.9300,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2022-02-24 11:00:00,73.860000,1.037217e+05,7676.300864,2565.230000,7283.009000,18707.780923,36866.000000,844.500240,31188.459796,84.6900,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


### Building a dataset / dataloader 

In [85]:
"""
df - dataframe with input variables,
labels - dataframe with output labels


"""

class lstm_dataset(Dataset):
    
    def __init__(self,df,labels):
        self.X = df.values
        self.labels = labels.values
        
    def __len__(self):
        return(len(self.X))
        
    def __getitem__(self,idx):    
        return(torch.tensor(self.X[idx]),torch.tensor(self.labels[idx]))

In [86]:
data_cols = []
lab_cols = []
for c in df_abes_cls.columns:
    if 'label' in c:
        lab_cols.append(c)
    else:
        data_cols.append(c)

In [87]:
df = df_abes_cls[data_cols]
labels = df_abes_cls[lab_cols]

In [88]:
dset = lstm_dataset(df,labels)

In [89]:
train_dataloader = DataLoader(dset, batch_size=64, shuffle= False)

In [198]:
tscv = TimeSeriesSplit()

In [201]:
a,b = next(iter(tscv.split(df_abes_cls)))

In [202]:
a

array([   0,    1,    2, ..., 4160, 4161, 4162])

In [206]:
df_abes_cls.iloc[b]

,close_avax,volume_avax,vol_usdt_K_avax,close_eth,volume_eth,vol_usdt_K_eth,close_btc,volume_btc,vol_usdt_K_btc,close_sol,...,avax_label_3,eth_label_1,eth_label_2,eth_label_3,btc_label_1,btc_label_2,btc_label_3,sol_label_1,sol_label_2,sol_label_3
datetime,,,,,,,,,,,,,,,,,,,,,
2020-12-18 09:30:00,3.1807,3962.80,12.563794,639.61,28393.63757,18094.331564,22764.77,2061.102492,46771.933704,1.6453,...,0,0,0,0,0,0,0,0,0,0
2020-12-18 10:00:00,3.2093,5581.26,17.825647,649.15,37266.53564,23993.306706,23059.34,2363.912977,54158.953887,1.6716,...,0,0,0,0,0,0,0,0,0,0
2020-12-18 10:30:00,3.1944,14076.81,44.987624,645.90,24178.26683,15638.154570,22988.21,1791.169418,41291.067823,1.6697,...,0,0,0,0,0,0,0,0,0,0
2020-12-18 11:00:00,3.2083,5933.42,19.028874,651.96,27995.05896,18207.578729,23193.60,1897.745141,43840.993528,1.6702,...,0,0,0,0,0,0,0,0,0,0
2020-12-18 11:30:00,3.2176,11496.26,36.945752,650.54,21168.49981,13806.301731,23003.31,1947.200331,45061.788486,1.6725,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-14 22:00:00,30.6542,76076.96,2336.500277,1869.76,11510.66644,21575.606525,60155.08,1154.298728,69570.230902,14.5822,...,0,0,0,0,0,0,0,0,0,0
2021-03-14 22:30:00,30.3979,70586.63,2145.166184,1864.46,14447.71070,26917.432283,59940.00,1388.781155,83205.769476,14.4998,...,0,0,0,0,0,0,0,0,0,0
2021-03-14 23:00:00,30.7338,47681.93,1464.417925,1874.99,9826.50537,18408.497763,60180.98,991.710474,59670.722295,14.5042,...,0,0,0,0,0,0,0,0,0,0


In [90]:
a,b = next(iter(train_dataloader))

In [130]:
df_abes_cls

,close_avax,volume_avax,vol_usdt_K_avax,close_eth,volume_eth,vol_usdt_K_eth,close_btc,volume_btc,vol_usdt_K_btc,close_sol,...,avax_label_3,eth_label_1,eth_label_2,eth_label_3,btc_label_1,btc_label_2,btc_label_3,sol_label_1,sol_label_2,sol_label_3
datetime,,,,,,,,,,,,,,,,,,,,,
2020-09-22 16:00:00,4.931200,1.914636e+06,9136.249619,339.990000,23133.333170,7898.110882,10412.110000,1147.850269,11969.341525,2.7540,...,0,0,0,0,0,0,0,0,1,1
2020-09-22 16:30:00,4.909600,1.256377e+06,6213.493286,336.120000,48515.721940,16362.884815,10372.900000,1801.381449,18703.921799,2.6728,...,0,0,0,0,0,0,0,0,1,1
2020-09-22 17:00:00,5.395100,1.100529e+06,5757.581164,339.580000,32510.296040,11019.057797,10429.770000,1388.738454,14468.525827,2.8134,...,0,0,0,0,0,0,0,0,1,1
2020-09-22 17:30:00,6.821900,2.783453e+06,17595.535036,342.110000,23494.623470,8002.336092,10463.180000,1288.599265,13468.384675,2.7602,...,0,0,0,0,0,0,0,0,1,1
2020-09-22 18:00:00,6.448800,2.538824e+06,16273.012136,340.500000,18641.707730,6364.471484,10445.040000,990.685762,10355.010636,2.7265,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-24 10:00:00,73.280000,8.301071e+04,6109.125097,2550.310000,13215.990300,33837.032941,36702.300000,1095.436680,40307.563870,84.3000,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2022-02-24 10:30:00,74.300000,6.866388e+04,5086.607976,2570.800000,10013.189800,25674.415957,36949.630000,900.574340,33227.209216,84.9300,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2022-02-24 11:00:00,73.860000,1.037217e+05,7676.300864,2565.230000,7283.009000,18707.780923,36866.000000,844.500240,31188.459796,84.6900,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [4]:
input_dim = 1
embedding_dim = 3

lstm = nn.LSTM(16,32)

NameError: name 'nn' is not defined

In [2]:
a1

NameError: name 'a1' is not defined

In [129]:
a1.float()

tensor([[[ 4.9312e+00,  1.9146e+06,  9.1362e+03,  ..., -6.0225e-03,
          -1.0707e-03, -1.6429e-02]],

        [[ 4.9096e+00,  1.2564e+06,  6.2135e+03,  ..., -1.1383e-02,
          -3.7658e-03, -2.9484e-02]],

        [[ 5.3951e+00,  1.1005e+06,  5.7576e+03,  ...,  1.0294e-02,
           5.4826e-03,  5.2604e-02]],

        ...,

        [[ 4.4096e+00,  3.6941e+05,  1.6628e+03,  ..., -7.6166e-04,
          -1.6797e-03, -2.0676e-03]],

        [[ 4.3010e+00,  3.2713e+05,  1.4095e+03,  ..., -4.9839e-03,
          -1.7902e-03, -1.9326e-02]],

        [[ 4.2242e+00,  2.0100e+05,  8.6272e+02,  ..., -8.2499e-04,
          -4.9346e-04, -5.2298e-03]]])

In [102]:
a1 = a.view(64,1,-1)

In [127]:
A = lstm(a1)

RuntimeError: expected scalar type Double but found Float

In [123]:
A[1][0].shape

torch.Size([1, 1, 12])

In [126]:
A[1][1]

tensor([[[ -1.,  14.,   1.,   0.,   1.,   0.,  -1.,   3., -54., -21.,  -1.,
          -64.]]], grad_fn=<StackBackward0>)

In [66]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

In [105]:
inputs

[tensor([[0.1040, 1.5941, 0.7651]]),
 tensor([[ 1.4222, -1.2603,  0.8249]]),
 tensor([[-0.6857, -1.7289, -0.7024]]),
 tensor([[-0.2533,  0.5306,  0.1989]]),
 tensor([[-2.7991,  0.3722, -1.1562]])]

In [68]:
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

In [69]:
hidden

(tensor([[[-0.2100, -0.6452,  1.5247]]]),
 tensor([[[-1.8982, -1.3277, -1.0177]]]))

In [70]:
inputs[0].view(1, 1, -1)

tensor([[[0.1040, 1.5941, 0.7651]]])

In [71]:
lstm(inputs[0].view(1, 1, -1))

(tensor([[[-0.1135, -0.2472, -0.3013]]], grad_fn=<StackBackward0>),
 (tensor([[[-0.1135, -0.2472, -0.3013]]], grad_fn=<StackBackward0>),
  tensor([[[-0.2507, -0.4064, -0.3817]]], grad_fn=<StackBackward0>)))

In [72]:
lstm(inputs[1].view(1, 1, -1))

(tensor([[[ 0.2541, -0.0257, -0.1554]]], grad_fn=<StackBackward0>),
 (tensor([[[ 0.2541, -0.0257, -0.1554]]], grad_fn=<StackBackward0>),
  tensor([[[ 0.3958, -0.1378, -0.2305]]], grad_fn=<StackBackward0>)))

In [73]:
lstm(inputs[2].view(1, 1, -1))

(tensor([[[-0.0016,  0.1102, -0.1095]]], grad_fn=<StackBackward0>),
 (tensor([[[-0.0016,  0.1102, -0.1095]]], grad_fn=<StackBackward0>),
  tensor([[[-0.0032,  0.2549, -0.1873]]], grad_fn=<StackBackward0>)))

In [76]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [77]:
training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]


In [79]:
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [74]:
inputs2 = torch.cat(inputs).view(len(inputs),1,-1)

In [81]:
lstm(inputs2,hidden)[0]

torch.Size([5, 1, 3])

In [65]:
inputs2

tensor([[[ 0.3752, -0.9313,  1.2965]],

        [[ 0.1018,  0.4679, -0.0287]],

        [[ 0.4414, -0.3677, -0.0606]],

        [[-1.1896, -0.0661,  0.7804]],

        [[ 1.4134, -0.7716,  0.1996]],

        [[-0.0482,  1.0321,  1.6705]],

        [[-1.5519, -1.0230,  0.1935]],

        [[-0.3985,  0.4188, -0.8607]],

        [[-0.8241, -0.7234, -1.0889]]])

In [197]:
next(iter(train_dataloader))

[tensor([[ 4.9312e+00,  1.9146e+06,  9.1362e+03,  ..., -6.0225e-03,
          -1.0707e-03, -1.6429e-02],
         [ 4.9096e+00,  1.2564e+06,  6.2135e+03,  ..., -1.1383e-02,
          -3.7658e-03, -2.9484e-02],
         [ 5.3951e+00,  1.1005e+06,  5.7576e+03,  ...,  1.0294e-02,
           5.4826e-03,  5.2604e-02],
         ...,
         [ 4.4096e+00,  3.6941e+05,  1.6628e+03,  ..., -7.6166e-04,
          -1.6797e-03, -2.0676e-03],
         [ 4.3010e+00,  3.2713e+05,  1.4095e+03,  ..., -4.9839e-03,
          -1.7902e-03, -1.9326e-02],
         [ 4.2242e+00,  2.0100e+05,  8.6272e+02,  ..., -8.2499e-04,
          -4.9346e-04, -5.2298e-03]], dtype=torch.float64),
 tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
    

True

In [170]:
torch.tensor(df_abes_cls.values[0])

tensor([ 4.9312e+00,  1.9146e+06,  9.1362e+03,  3.3999e+02,  2.3133e+04,
         7.8981e+03,  1.0412e+04,  1.1479e+03,  1.1969e+04,  2.7540e+00,
         5.3219e+04,  1.4812e+02,  1.0264e-02, -6.0225e-03, -1.0707e-03,
        -1.6429e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  1.0000e+00,  1.0000e+00], dtype=torch.float64)

### Simple LSTM model

In [183]:
#(df_abes.loc[bad_indices[1] - timedelta(minutes = 30):bad_indices[1]])

In [184]:
#df_abes_cls.loc[bad_indices[0] - timedelta(minutes = 30)]

In [25]:
close_cols

['close_avax', 'close_eth', 'close_btc', 'close_sol']

In [ ]:
def get_rets

In [39]:
df_abels.loc[idx_30t[0]]['close_avax'] - df_abels.loc[z]['close_avax']

0.5547000000000004

In [47]:
idx_int_1 = list(set(idx).intersection(set([x + timedelta(days = 1) for x in idx_int])))

In [48]:
len(idx_int_1)

24818

In [45]:
df_abels.loc[idx_int]['close_avax'] - df_abels.loc[[x + timedelta(days = 1) for x in idx_int]]['close_avax']

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: DatetimeIndex(['2021-04-25 17:30:00', '2021-04-25 16:30:00',\n               '2021-03-06 12:00:00', '2022-02-24 23:00:00',\n               '2021-04-25 16:00:00',\n               ...\n               '2022-02-25 10:30:00', '2021-04-20 12:30:00',\n               '2021-09-29 17:00:00', '2021-08-13 13:30:00',\n               '2020-12-25 11:30:00'],\n              dtype='datetime64[ns]', name='datetime', length=91, freq=None). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [40]:
df_abels.loc[idx_30t[0]]

open_avax              4.880000
high_avax              4.933600
low_avax               4.834800
close_avax             4.928300
volume_avax        40984.580000
vol_usdt_K_avax      200.994528
open_eth             342.050000
high_eth             342.860000
low_eth              341.970000
close_eth            342.830000
volume_eth          1392.861650
vol_usdt_K_eth       476.971543
open_btc           10423.270000
high_btc           10436.260000
low_btc            10423.260000
close_btc          10433.550000
volume_btc            83.407394
vol_usdt_K_btc       869.806502
open_luna              0.293400
high_luna              0.293400
low_luna               0.293400
close_luna             0.293400
volume_luna            0.000000
vol_usdt_K_luna        0.000000
open_sol               2.800000
high_sol               2.819600
low_sol                2.787400
close_sol              2.812400
volume_sol          2839.890000
vol_usdt_K_sol         7.969299
Name: 2020-09-22 16:00:00, dtype: float6

In [33]:
z = idx_30t[0] + timedelta(days = 1)

df_abels.loc[z]

open_avax              4.356700
high_avax              4.387600
low_avax               4.355200
close_avax             4.373600
volume_avax         2508.320000
vol_usdt_K_avax       10.949193
open_eth             338.420000
high_eth             338.440000
low_eth              338.000000
close_eth            338.400000
volume_eth           866.970950
vol_usdt_K_eth       293.391639
open_btc           10436.490000
high_btc           10436.490000
low_btc            10428.140000
close_btc          10434.370000
volume_btc            42.407284
vol_usdt_K_btc       442.538244
open_luna              0.293800
high_luna              0.293800
low_luna               0.293800
close_luna             0.293800
volume_luna            0.000000
vol_usdt_K_luna        0.000000
open_sol               2.830500
high_sol               2.830500
low_sol                2.830500
close_sol              2.830500
volume_sol             0.000000
vol_usdt_K_sol         0.000000
Name: 2020-09-23 16:00:00, dtype: float6

In [21]:
len(idx_int)

24909

In [24]:
df_abels.loc[idx_int]

,open_avax,high_avax,low_avax,close_avax,volume_avax,vol_usdt_K_avax,open_eth,high_eth,low_eth,close_eth,...,low_luna,close_luna,volume_luna,vol_usdt_K_luna,open_sol,high_sol,low_sol,close_sol,volume_sol,vol_usdt_K_sol
datetime,,,,,,,,,,,,,,,,,,,,,
2022-02-14 12:00:00,76.5000,76.6600,76.4900,76.5100,403.580,30.875888,2844.48,2847.42,2842.20,2842.21,...,51.8700,51.8700,2584.180,134.144784,91.6200,91.7600,91.5700,91.5900,536.770,49.170816
2021-11-22 05:00:00,131.3700,131.6200,131.1300,131.4400,3668.870,482.107862,4345.99,4353.48,4344.64,4349.63,...,43.4900,43.5600,2112.710,91.987393,228.8400,229.7100,228.7200,229.5700,1243.840,285.094347
2022-01-09 04:30:00,80.0800,80.3900,79.9700,80.3100,1637.180,131.293650,3019.00,3027.50,3016.04,3026.00,...,63.2800,63.5500,9280.390,589.119157,134.2500,134.8300,134.0600,134.6300,3098.690,416.587884
2020-11-23 16:00:00,4.1498,4.1500,4.1498,4.1500,198.950,0.825623,586.24,587.73,585.59,585.59,...,0.3520,0.3529,12153.820,4.286045,2.3142,2.3195,2.2998,2.2999,3048.640,7.033365
2021-10-22 22:30:00,66.1500,66.2000,66.0100,66.0700,10031.590,663.188415,4114.22,4118.92,4114.21,4116.31,...,45.0400,45.0900,4570.820,206.258252,212.8900,212.9000,212.0700,212.5500,14408.760,3065.031427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-22 16:00:00,11.2480,11.2570,11.2350,11.2540,838.205,9.430644,1950.27,1950.50,1946.93,1949.40,...,5.1890,5.1950,940.839,4.888129,26.8710,26.8940,26.7370,26.7500,3796.164,101.777055
2021-06-06 23:00:00,17.4890,17.4890,17.4500,17.4750,145.825,2.549313,2660.00,2661.52,2655.00,2661.52,...,6.0650,6.0650,16597.416,100.804406,40.7170,40.7740,40.3490,40.4000,39787.039,1613.702621
2021-03-13 08:30:00,28.2900,28.2929,28.2200,28.2673,716.740,20.268440,1767.53,1768.30,1765.82,1767.18,...,13.6787,13.7108,10916.220,149.551668,14.5157,14.5349,14.5100,14.5349,544.010,7.901908


In [23]:
len(df_abels.loc[idx_int])

24909

In [28]:
c.loc[1613044260000]

,open_avax,high_avax,low_avax,close_avax,volume_avax,pct_change_avax,log_ret_avax,vol_usdt_K_avax,open_eth,high_eth,low_eth,close_eth,volume_eth,pct_change_eth,log_ret_eth,vol_usdt_K_eth
timestamp,,,,,,,,,,,,,,,,
1613044260000,48.9746,49.4186,48.9746,49.2099,3152.87,0.00578,0.005763,154.781482,1776.44,1778.11,1772.1,1773.13,1075.74153,-0.001813,-0.001814,1909.209931
1613044260000,48.9746,49.4186,48.9746,49.2099,3152.87,0.00578,0.005763,154.781482,1776.44,1778.11,1772.1,1773.13,1075.74153,-0.001813,-0.001814,1909.209931
1613044260000,48.9746,49.4186,48.9746,49.2099,3152.87,0.00578,0.005763,154.781482,1776.44,1778.11,1772.1,1773.13,1075.74153,-0.001813,-0.001814,1909.209931
1613044260000,48.9746,49.4186,48.9746,49.2099,3152.87,0.00578,0.005763,154.781482,1776.44,1778.11,1772.1,1773.13,1075.74153,-0.001813,-0.001814,1909.209931


In [20]:
df_avax.index.values_counts()

AttributeError: 'DatetimeIndex' object has no attribute 'values_counts'

In [42]:
a = pd.Series(df_luna.index).value_counts()

In [73]:
df_avax.loc[1613043420000]

,open_avax,high_avax,low_avax,close_avax,volume_avax,pct_change_avax,log_ret_avax,vol_usdt_K_avax
timestamp,,,,,,,,
1613043420000,50.2547,50.2624,50.0565,50.1,817.85,-0.003731,-0.003738,41.037546
1613043420000,50.2547,50.2624,50.0565,50.1,817.85,-0.003731,-0.003738,41.037546


In [43]:
count = 0
for i in a.index:
    if a[i] > 1:
        count += a[i]
        
        
count

0

In [60]:
b = list(df_avax.index)

In [62]:
b = list(set(b))
print(len(b))
b.sort()

747243


In [63]:
df_avax = df_avax.loc[b]

In [70]:
len(df_avax)

748998

In [82]:
df_avax.columns

Index(['open_avax', 'high_avax', 'low_avax', 'close_avax', 'volume_avax',
       'pct_change_avax', 'log_ret_avax', 'vol_usdt_K_avax'],
      dtype='object')

In [84]:
df_avax['timestamp'] = df_avax.index

In [85]:
len(df_avax.drop_duplicates(subset = 'timestamp'))

747243

In [67]:
len(df_avax)

720399

In [79]:
len(df_avax.index) - len(set(df_avax.index))

1755

In [86]:
748998 - 747243

1755